In [1]:
import tensorflow as tf
import datetime
import os
from tensorflow.keras import layers

In [2]:
str_contant = tf.constant('this is string')

In [3]:
print(tf.add(1,2))
print(tf.add([1,2], [3,4]))
print(tf.square(5))
print(tf.reduce_sum([1,2,3,4,5]))
print(tf.reduce_sum([[1,2,3], [4,5,6], [7,8,9]], axis = 1))

tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor([4 6], shape=(2,), dtype=int32)
tf.Tensor(25, shape=(), dtype=int32)
tf.Tensor(15, shape=(), dtype=int32)
tf.Tensor([ 6 15 24], shape=(3,), dtype=int32)


In [4]:
x = tf.matmul([[2]], [[2,3]])

In [5]:
x

<tf.Tensor: shape=(1, 2), dtype=int32, numpy=array([[4, 6]], dtype=int32)>

## Numpy compatibility

In [6]:
import numpy as np

arr = np.ones([3,3])
tensor = tf.multiply(arr, 42)
tensor

<tf.Tensor: shape=(3, 3), dtype=float64, numpy=
array([[42., 42., 42.],
       [42., 42., 42.],
       [42., 42., 42.]])>

In [7]:
print(np.add(tensor, 1))
tensor.numpy()

[[43. 43. 43.]
 [43. 43. 43.]
 [43. 43. 43.]]


array([[42., 42., 42.],
       [42., 42., 42.],
       [42., 42., 42.]])

## GPU

In [8]:
print(tf.config.experimental.list_physical_devices())

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


In [9]:
if tf.config.experimental.list_physical_devices('GPU'):
    print('GPU is available')
    with tf.device("GPU:0"): #name of gpu device
        x = tf.random.uniform([1000,1000])
        assert x.device.endwith("GPU:0")
        x1 = x * x
else:
    x = tf.random.uniform([1000,1000])
    x1 = x * x    

## Datasets

In [10]:
ds_tensor = tf.data.Dataset.from_tensor_slices([1,2,3,4,5,6])

In [11]:
for elem in ds_tensor:
    print(elem.numpy())

1
2
3
4
5
6


In [12]:
ds_tensor = ds_tensor.map(tf.square).shuffle(4).batch(2)

In [13]:
for i in ds_tensor:
    print(i)

tf.Tensor([16  9], shape=(2,), dtype=int32)
tf.Tensor([4 1], shape=(2,), dtype=int32)
tf.Tensor([36 25], shape=(2,), dtype=int32)


In [14]:
# create csv
import tempfile
_, name = tempfile.mkstemp()
with open(name, 'w+') as file:
    file.write("""line 1
    line 2
    line 3
    line 4
    line 5
    line 6
    line 7
    line 8
    line 9    """)

ds_file = tf.data.TextLineDataset(name)

In [15]:
ds_file = ds_file.batch(3)

In [16]:
for i in ds_file:
    print(i)

tf.Tensor([b'line 1' b'    line 2' b'    line 3'], shape=(3,), dtype=string)
tf.Tensor([b'    line 4' b'    line 5' b'    line 6'], shape=(3,), dtype=string)
tf.Tensor([b'    line 7' b'    line 8' b'    line 9    '], shape=(3,), dtype=string)


## Session

In [17]:
tf.compat.v1.disable_eager_execution()
const = tf.constant(1.0)

In [18]:
with tf.compat.v1.Session() as sess: # first way
    output = sess.run(const)
    print(output)

1.0


In [19]:
x = tf.constant([1,2,3,4,5])
y = tf.constant([6,7,8,9,10])

In [20]:
result = tf.multiply(x,y)
result

<tf.Tensor 'Mul:0' shape=(5,) dtype=int32>

In [21]:
sess = tf.compat.v1.Session() # second way
print(sess.run(result))
sess.close()

[ 6 14 24 36 50]


## Variables

In [22]:
a = tf.Variable(1, name='scalar')
b = tf.Variable([1,2,3], name='vector')
t_2 = tf.Variable([[1,2,3], [1,1,0], [1,0,1]], name='matrix')

print(a, a.get_shape())
print()
print(b, b.get_shape())
print()
print(t_2, t_2.get_shape())

<tf.Variable 'scalar:0' shape=() dtype=int32> ()

<tf.Variable 'vector:0' shape=(3,) dtype=int32> (3,)

<tf.Variable 'matrix:0' shape=(3, 3) dtype=int32> (3, 3)


In [23]:
x = tf.Variable(1)
y = tf.Variable(10)

op1 = tf.add(x,y) #sum

init = tf.compat.v1.variables_initializer([x, y], name='init')

with tf.compat.v1.Session() as sess:
    sess.run(init)
    print(sess.run(op1))

11


In [24]:
x = [[2]]
m = tf.matmul(x, x)

with tf.compat.v1.Session() as sess:
    sess.run(init)
    print(sess.run(op1))

11


## Placeholders

In [25]:
x = tf.compat.v1.placeholder(tf.float32, shape=[1024, 1024]) #place for variable

y = tf.matmul(x,x) # define operation

with tf.compat.v1.Session() as sess:
    rand_array = np.random.rand(1024, 1024)
    
    print(sess.run(y, feed_dict = {x: rand_array}))

[[246.7768  255.97011 251.92921 ... 255.62819 253.86455 243.63939]
 [250.34698 260.0139  259.98282 ... 260.1554  251.20975 246.23239]
 [251.86276 269.47427 262.47955 ... 258.68457 261.7362  255.15141]
 ...
 [248.08871 261.15378 255.8465  ... 257.8009  258.4054  243.75475]
 [260.90576 270.80725 266.08023 ... 269.7939  260.19833 259.59372]
 [258.63242 264.997   265.5297  ... 264.4882  262.36356 258.1708 ]]


## Tensorboard

In [26]:
tf.compat.v1.reset_default_graph() # reset variables above

In [27]:
x = tf.constant(1.0, name='input')

w = tf.Variable(0.8, name='weight')

y = tf.math.multiply(w, x, name='output')

In [28]:
init = tf.compat.v1.global_variables_initializer()
sess1 = tf.compat.v1.Session()
sess1.run(init)

In [29]:
sess1.run(y)

0.8

In [30]:
%load_ext tensorboard

In [31]:
logdir = os.path.join('logs', datetime.datetime.now().strftime("%Y%m%d - %H%M%S"))
summary_writer = tf.compat.v1.summary.FileWriter(logdir, sess1.graph)

In [32]:
%tensorboard --logdir logs

In [33]:
sess1.close()

## Train model

In [34]:
tf.compat.v1.reset_default_graph()

In [35]:
inp = tf.constant([1.2, 3.0, -2.2])

weight = tf.Variable([1., 2., 3.], dtype=tf.float32)

out_true = tf.constant(0.)
out_pred = tf.reduce_sum(inp*weight)

loss = (out_pred - out_true)**2

init = tf.compat.v1.global_variables_initializer()

sess = tf.compat.v1.Session()

optim = tf.compat.v1.train.GradientDescentOptimizer(learning_rate = 0.025)

compute_grad = optim.co

AttributeError: 'GradientDescentOptimizer' object has no attribute 'co'